In [27]:
from pyspark import SparkContext, SparkConf 
from pyspark.sql import SparkSession
import pyspark.sql as sql
from pyspark.sql.functions import col

In [63]:
conf = SparkConf().setAppName("my_LR_1").setMaster('local[1]')
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

In [64]:
sc# проверка

<SparkContext master=local[1] appName=my_LR_1>

In [65]:
!hadoop fs -ls

Found 3 items
-rwxr-xr-x   3 kuricyn kuricyn      12237 2023-12-20 13:46 Untitled.ipynb
-rwxr-xr-x   3 kuricyn kuricyn       5647 2023-12-20 13:46 station.csv
-rwxr-xr-x   3 kuricyn kuricyn   80208848 2023-12-20 13:46 trip.csv


Видно, что загружено три файла

In [95]:
# Чтение данных из файла "trip.csv" и создание DataFrame trip_data
trip_data = spark.read\
.option("header", True)\
.option("inferSchema", True)\
.option("timestampFormat", 'M/d/y H:m')\
.csv("trip.csv")
# Чтение данных из файла "station.csv" и создание DataFrame station_data
staation_data = spark.read\
.option("header", True)\
.option("inferSchema", True)\
.option("timestampFormat", 'M/d/y')\
.csv("station.csv")

№Задание 1. Найти велосипед с максимальным временем пробега (буду делать через SQL)

In [96]:
trip_data.createOrReplaceTempView("trip")
# Выполнение SQL-подобного запроса
result = spark.sql("SELECT bike_id, sum(duration) as dur from trip group by bike_id order by dur DESC LIMIT 1")
print("Результат:")
result.show()

Результат:
+-------+--------+
|bike_id|     dur|
+-------+--------+
|    535|18611693|
+-------+--------+



Задание 2. Найти наибольшее геодезическое расстояние между станциями.

In [78]:
# Создание нового DataFrame 
joined_station = staation_data.crossJoin(staation_data)

In [79]:
from math import radians, cos, sin, asin, sqrt

def calculate_distance(lat1, lon1, lat2, lon2):
    # Преобразуем координаты в радианы
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Применяем формулу Хаверсина
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371 # Радиус Земли в километрах
    return c * r

In [80]:
# Использование метода map для применения функции calculate_distance
result_2 = joined_station.rdd.map(lambda x: calculate_distance(x[2], x[3], x[9], x[10])).max()

In [83]:
print(f"Наибольшее геодезическое расстояние между станциями: {result_2:.2f}")

Наибольшее геодезическое расстояние между станциями: 69.92


Задание 3. Найти путь велосипеда с максимальным временем пробега через станции.

In [87]:
temp = trip_data.orderBy(col("duration").desc()).limit(1).collect()
result_3 = joined_station.rdd.filter(lambda l: (l[1] == temp[0].start_station_name) and (l[8] == temp[0].end_station_name))\
.map(lambda x: calculate_distance(x[2], x[3], x[9], x[10]))
print(f"путь велосипеда с максимальным временем пробега через станции: {result_3.collect()}")

путь велосипеда с максимальным временем пробега через станции: [2.312047985749405]


Задание 4. Найти количество велосипедов в системе.

In [89]:
result_4 = spark.sql("select count(distinct bike_id) from trip")
print("Результат:")
result_4.show()

Результат:
+-----------------------+
|count(DISTINCT bike_id)|
+-----------------------+
|                    700|
+-----------------------+



Задание 5. Найти пользователей потративших на поездки более 3 часов.

In [94]:
result_5 = spark.sql("SELECT id, sum_duration FROM ( SELECT id, sum(duration) as sum_duration FROM trip GROUP BY id) WHERE sum_duration > 10800")
print("Результат:")
result_5.show()

Результат:
+------+------------+
|    id|sum_duration|
+------+------------+
|  6654|       17751|
| 22097|       21686|
| 22223|       15619|
| 30654|       13479|
| 34759|       17959|
| 43688|       22504|
| 88666|       21964|
| 88674|       13726|
|105536|       19854|
|143153|       20649|
|146988|       44084|
|189310|       21785|
|431881|       28377|
|431018|       12301|
|427387|       12612|
|418759|       15526|
|418461|       15103|
|410754|       16743|
|386707|       14313|
|305619|       12412|
+------+------------+
only showing top 20 rows



In [62]:
sc.stop()# отключаемся